# Introduction #

In this notebook we'll go through a start-to-finish example of building a dataset of TFRecords. To read the image files, we'll use a Keras dataset constructor new in TF2.3. You could read the data another way, if you like, but this simplifies some of the preprocessing.

In [1]:
# tf.keras.preprocessing.image_dataset_from_directory will be available in TF2.3
!pip install -q tf-nightly

import tensorflow as tf

# Load Image Data #

In [2]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True,
                                   cache_dir="/kaggle/working/downloaded")
data_dir = pathlib.Path(data_dir)

In [3]:
from functools import partial

IMG_HEIGHT = 512
IMG_WIDTH = 512

load_split = partial(
    tf.keras.preprocessing.image_dataset_from_directory,
    data_dir,
    validation_split=0.2,
    shuffle=True,
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
)

ds_train = load_split(subset='training')
ds_valid = load_split(subset='validation')

class_names = ds_train.class_names
print("\nClass names: {}".format(class_names))

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.

Class names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


2022-02-27 08:29:45.243530: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Wrap with tf.Example #

In [4]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

def process_image(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
    image = tf.io.encode_jpeg(image)
    return image, label

ds_train_encoded = (
    ds_train
    .unbatch()
    .map(process_image)
)

ds_valid_encoded = (
    ds_valid
    .unbatch()
    .map(process_image)
)

In [5]:
def make_example(encoded_image, label):
    image_feature = Feature(
        bytes_list=BytesList(value=[
            encoded_image,
        ]),
    )
    label_feature = Feature(
        int64_list=Int64List(value=[
            label,
        ])
    )

    features = Features(feature={
        'image': image_feature,
        'label': label_feature,
    })
    
    example = Example(features=features)
    
    return example.SerializeToString()

To shard the dataset, we'll use `tf.data.Datset.shard`. We'll write the TFRecords by iterating shard by shard in an outer loop and within each shard in an inner loop. For TPU training, you'll want at least 8 shards for each split. First the training set:

In [8]:
!mkdir '/kaggle/working/training'

NUM_SHARDS = 32
PATH = '/kaggle/working/training/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        ds_train_encoded
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

mkdir: /kaggle/working: No such file or directory


NotFoundError: /kaggle/working/training/shard_00.tfrecord; No such file or directory

Now the validation set:

In [ ]:
!mkdir '/kaggle/working/validation'

NUM_SHARDS = 8
PATH = '/kaggle/working/validation/shard_{:02d}.tfrecord'

for shard in range(NUM_SHARDS):
    ds_shard = (
        ds_valid_encoded
        .shard(NUM_SHARDS, shard)
        .as_numpy_iterator()
    )
    with tf.io.TFRecordWriter(path=PATH.format(shard)) as f:
        for encoded_image, label in ds_shard:
            example = make_example(encoded_image, label)
            f.write(example)

Finally, we'll clean up the downloaded files.

In [ ]:
!rm -rf '/kaggle/working/downloaded'

If the dataset isn't too big, you could build the TFRecords within a notebook here on Kaggle and create a new dataset directly from the notebook. (Do *Save Version*, *Save & Run All*, and then create a new dataset from the notebook output.) For larger datasets, you could either run it on a personal machine or upgrade to a Cloud AI notebook. (See the option in the *File* menu.)